In [1]:
import numpy as np
import tensorflow as tf

<ul>
    <li>central unit of data = tensor</li>
    <li>tensor = n-dimensional array</li>
    <li>rank of tensor = number of dimensions</li>
    <li>shape of tensor = tuple specifying array's length in each dimension</li>
</ul>
TensorFlow uses numpy arrays to represent tensor values.

In [2]:
3. # a rank 0 tensor; a scalar with shape [],
[1., 2., 3.] # a rank 1 tensor; a vector with shape [3]
[[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3]
[[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]

[[[1.0, 2.0, 3.0]], [[7.0, 8.0, 9.0]]]

<h2>TF Workflow</h2>
<ol>
    <li>Build a computational graph (tf.Graph)</li>
    <li>Run the computational graph (tf.Session)</li>
</ol>

<h2>TF Graph</h2>
A computational graph is a series of TF operations
<ul>
    <li><b>Nodes</b> of the graph represent operations (add, subtract, multiply)</li>
    <li><b>Edges</b> of the graph represent the values (tensors) that flow through the graph</li>
</ul>

In [3]:
# create two floating point tf constants
a = tf.constant(3.0, tf.float32)
b = tf.constant(4.0) # float32 is implicit here

# adding the two contants
total = tf.add(a, b)

print a
print b
print total

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("Add:0", shape=(), dtype=float32)


<p>The above statements build only a computational graph. These <b>tf.Tensor</b> objects just represent the results of the operations that will be run.</p>
<p>
    Each operation in a graph is given a unique name. It is independent of the name of the object. Tensors are named after the operation the produces them followed by an output index as in <b>Add:0</b>.
</p>

<h2>TensorBoard</h2>
<p>TensorBoard can be used to visualize a computational graph</p>
<p>Save the computation graph to a TensorBoard summary file</p>

In [4]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

<p>These will produce an <b>event</b> file in the current directory with the format <b>events.out.tfevents.{timestamp}.{hostname}</b>.</p>
<p>
    Now, in a new terminal, launch TensorBoard with the following shell command:
</p>
<b>tensorboard --logdir .</b>

<h2>Session</h2>
<p>To evaluate tensors, create <b>tf.Session</b> object. It encapsulates TF runtime and runs TF operations.</p>

In [6]:
sess = tf.Session()
print sess.run(total)

# you can pass multiple tensors to run(). handles any combination of tuples or dictionaries
print sess.run({'ab':(a, b), 'total':total})

7.0
{'total': 7.0, 'ab': (3.0, 4.0)}


<h2>Feeding</h2>
<p>A graph can be made to accept external inputs using <b>placeholders</b>.</p>

In [7]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y # + operator is overloaded to add tensors


pass the placeholders tensors through <b>feed_dict</b> argument of <b>run()</b> method

In [8]:

print sess.run(z, feed_dict={x: 3, y: 4})
print sess.run(z, feed_dict={x: [1, 2], y: [3, 4]})

7.0
[ 4.  6.]


<h2>Datasets</h2>
<p>Placeholders work for simple programs. But Datasets are standard for streaming data into models.</p>

In [13]:
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]

# from_tensor_slices() creates a Dataset whose elements are slices of the given tensor
slices = tf.data.Dataset.from_tensor_slices(my_data)
print slices

# make_one_shot_iterator() returns an initialized iterator
next_item = slices.make_one_shot_iterator().get_next()
print next_item

<TensorSliceDataset shapes: (2,), types: tf.int32>
Tensor("IteratorGetNext_2:0", shape=(2,), dtype=int32)


<p>
    <b>from_tensor_slices()</b> creates a Dataset whose elements are slices of the given tensor.
</p>
<p>
    <b>make_one_shot_iterator()</b> Creates an Iterator for enumerating the elements of this dataset. The returned iterator will be initialized automatically.
</p>
<p>
    <b>get_next()</b> returns a nested structure of tf.Tensors representing the next element. You call this once and use its result as the input to another computation. Each time the result tensor of get_next() is run, the iterator will point to the next slice of the data. Reaching the end of the data stream causes Dataset to throw an <b>OutOfRangeError</b>.
</p>

In [14]:
while True:
    try:
        print(sess.run(next_item))
    except tf.errors.OutOfRangeError:
        break

[0 1]
[2 3]
[4 5]
[6 7]
